In [ ]:
from functools import reduce
from fuzix.sims.pendulumcart import draw_world
from fuzix.sims.pendulumcart import simulate
from fuzix.utils import do_render_animation
from fuzix.utils import render_animation
from fuzix.utils import render_animation_iter
from fuzix.utils import memoize
from fuzix.utils import RendererWidget
from fuzix.utils import timed
from fuzix.utils import renderer_scope
from fuzix.draw_utils import draw_circle
from fuzix.draw_utils import draw_rect
from fuzix.draw_utils import draw_line
from IPython.display import display
from IPython.display import HTML
from IPython.display import Markdown
from ipywidgets import interact
from matplotlib import pyplot as plt, cm, animation, rc
from matplotlib.lines import Line2D
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
import daglet
import functools
import ipywidgets
import matplotlib
import matplotlib.patches as patches
import numbers
import numpy as np
import operator
import sympy as sp
from fuzix.scene import Param
from fuzix.scene import Block
from fuzix.scene import State
from fuzix.scene import Rod
from fuzix.scene import Connector
from fuzix.scene import Scene
from fuzix.scene import Ball
from fuzix.scene import HingeRod
from fuzix.scene import World
from fuzix.scene import show_scene_viewer
from fuzix.scene import show_renderer
from fuzix.scene import Simulator
from fuzix.scene import Spring

In [ ]:
%matplotlib inline
rc('animation', html='html5')
#rc('animation', html='jshtml')

sp.init_printing(use_unicode=True)
init_vprinting()

In [ ]:
theta1 = State()
theta2 = State()
scene = Scene([
    Connector()
    | World()
    | Ball(mass=0.1)
    | HingeRod(angle=theta1)
    | Ball()
    | HingeRod(angle=theta2)
    | Ball()

])

params = {}
initial_state = {}
scale = 0.8

@interact(
    theta1_=(-np.pi, np.pi),
    theta2_=(-np.pi, np.pi),
)
def f(
        theta1_=1.8,
        theta2_=0.9,
):
    global params, initial_state
    params.update({
    })
    initial_state.update({
        theta1: theta1_,
        theta2: theta2_,
    })
    scene.draw(params=params, state=initial_state, scale=scale)

In [ ]:
simulator = Simulator(scene)

In [ ]:
params = {}
#initial_state = {
#    #angle1: 1.8,
#}

print(scene.Q)
l1 = 3.
l2 = 3.
m1 = 1.
m2 = 1.
g = 10.

def get_ddt2(t1, t2, dt1, dt2):
    return 1/(m2*l2**2-1/(m1+m2)*(m2*l1*l2*np.cos(t2-t1))) * (m2*l1*l2*dt1*(dt2-dt1)*np.sin(t2-t1)-m2*g*np.cos(t2)-1/(m1+m2)*(m2*l1*l2*np.cos(t2-t1))*(m2*l1*l2*dt2*(dt2-dt1)*np.sin(t2-t1)-(m1+m2)*g*np.cos(t1)))

def get_ddt1(t1, t2, dt1, dt2):
    ddt2 = get_ddt2(t1, t2, dt1, dt2)
    return 1/((m1+m2)*l1**2) * (m2*l1*l2*dt2*(dt2-dt1)*np.sin(t2-t1)-(m1+m2)*g*np.cos(t1) - m2*l1*l2*np.cos(t2-t1)*ddt2)

funcs = [
    #lambda theta1, theta2, dtheta1, dtheta2: m2*l2*np.sin(theta2-theta1)-GRAVITY/l1*np.sin(theta1),
    #lambda theta1, theta2, dtheta1, dtheta2: -m2*l1*np.sin(theta2-theta1)-GRAVITY/l2*np.sin(theta2),
    get_ddt1,
    get_ddt2,
]
funcs = None
states = simulator.run(params, initial_state, max_time=500, funcs=funcs)
show_scene_viewer(scene, params, states, scale=scale)
show_renderer(scene, params, states, scale=scale, sample_interval=6)

In [ ]:
#t = sp.Symbol('t')
#Q = [symbol for symbol, _ in scene.state_symbols]
#dQ = [q.diff(t) for q in Q]
#
#L = scene.get_energy_expr(t)
#
#display(Markdown('## Lagrangian:'))
#display(sp.Eq(sp.Symbol('L'), L))
#
#display(Markdown('## Lagrange Equations:'))
#eqs = [get_lagrange_eq(L, q, t) for q in Q]
#display({q.diff(t, 2): eq for q, eq in zip(Q, eqs)})

In [ ]:
#eqs = solve_lagrangian(L, Q, t)

In [ ]:
#params = {symbol: default_value for symbol, default_value in scene.param_symbols}
#funcs = get_funcs(Q, eqs, params)

In [ ]:
world = World()
track = world | TrackSpec()
block = track | BlockSpec()
hinge = block | HingeSpec()
rod = hinge | RodSpec()
ball = rod | BallSpec()

scene = Scene([ball2])

@interact(
    track_x=(-2., 15.),
    hinge_theta=(-2*np.pi, 2*np.pi),
)
def f(
    track_x=2.,
    rod_length=3.,
    hinge_theta=0.5,
):
    params = {
        rod.spec.length: rod_length,
        world.spec.gravity: 10.,
        block.spec.mass: 1.,
        ball.spec.mass: 1.,
    }
    state = {
        hinge.spec.theta: hinge_theta,
        track.spec.x: track_x,
        hinge.spec.theta.diff(t): 0.,
        track.spec.x.diff(t): 0.,
    }
    fig, ax = plt.subplots()
    scene.draw(ax, params, state)

In [ ]:
def fig2np(fig):
    fig.canvas.draw() 
    w, h = fig.canvas.get_width_height()
    buffer = fig.canvas.buffer_rgba()
    return (
        np
        .frombuffer(buffer, dtype=np.uint8)
        .reshape(h, w, 4)
    )


fig, ax = plt.subplots(dpi=100)
for x_ in np.linspace(0., 2., 5):
    params = {}
    initial_state = {
        x: x_,
    }
    scene.draw(ax, params=params, state=initial_state, scale=scale)
    ax.axis('off')
    image = fig2np(fig)

    ax.clear()
    
fig.clear()

fig, ax2 = plt.subplots(figsize=(16*.8, 9*.8))
ax2.imshow(image, aspect='equal', interpolation='hanning')
ax2.axis('off')
None

In [ ]:
image.shape

In [ ]:
np.stack([image, image, image]).shape

In [ ]:
def sample_iter(iterator, sample_interval):
    return islice(iterator, 0, None, sample_interval)


with timed() as timing:
    sample_interval = 5
    frames = []
    with renderer_scope(frames) as frame_scope:
        iterator = simulator.iterate(params, initial_state, max_time=300)
        for state in sample_iter(iterator, sample_interval):
            with frame_scope() as ax:
                scene.draw(ax, params=params, state=state, scale=scale)

timing[0], len(frames)

In [ ]:
with timed() as timing:
    states = list(simulator.iterate(params, initial_state, max_time=300))
    draw_func = lambda ax, time_index: scene.draw(ax, params, states[time_index], scale=scale)
    anim = do_render_animation(draw_func, len(states), sample_interval)
    anim.save('anim.mp4')

#display(anim)
timing[0]

In [ ]:

@interact(t=(0, len(frames) - 1))
def f(t=0):
    fig, ax = plt.subplots(figsize=(16*.8, 9*.8))
    ax.axis('off')
    ax.imshow(frames[t], aspect='equal', interpolation='hanning')